In [81]:
import spacy
import pandas as pd
import pickle
import numpy as np
from scipy import spatial

In [13]:
nlp = spacy.load("en_core_web_sm")

# 0. Read text data

In [14]:
text_data  = pd.read_pickle(r'../features/all_data_features_17_20.pkl')
print(text_data.shape)
text_data.head(3)

(4897, 26)


,cdate,content,ddate,details,forum,id,invitation,label,nonreaders,number,...,tmdate,writers,title,authors,abstract,keywords,tldr,ref_len,ref_latest,ref_years
2017_B1-Hhnslg,None,{'title': 'Prototypical Networks for Few-shot ...,None,{'replyCount': 9},B1-Hhnslg,2017_B1-Hhnslg,ICLR.cc/2017/conference/-/submission,Reject,[],581,...,1481060777491,[],Prototypical Networks for Few-shot Learning,"[Jake Snell, Kevin Swersky, Richard Zemel]",A recent approach to few-shot classification c...,"[Deep learning, Transfer Learning]",We learn a metric space in which few-shot clas...,26,2016,"[2013, 2015, 2013, 2009, 2016, 2004, 2014, 201..."
2017_B1-q5Pqxl,None,{'title': 'Machine Comprehension Using Match-L...,None,{'replyCount': 18},B1-q5Pqxl,2017_B1-q5Pqxl,ICLR.cc/2017/conference/-/submission,Accept,[],417,...,1489370977828,[],Machine Comprehension Using Match-LSTM and Ans...,"[Shuohang Wang, Jing Jiang]",Machine comprehension of text is an important ...,"[Natural language processing, Deep learning]",Using Match-LSTM and Answer Pointer to select ...,24,2016,"[2015, 2016, 2016, 2016, 2016, 2015, 2016, 201..."
2017_B16dGcqlx,None,"{'title': 'Third Person Imitation Learning', '...",None,{'replyCount': 14},B16dGcqlx,2017_B16dGcqlx,ICLR.cc/2017/conference/-/submission,Accept,[],531,...,1488760091458,[],Third Person Imitation Learning,"[Bradly C Stadie, Pieter Abbeel, Ilya Sutskever]",Reinforcement learning (RL) makes it possible ...,[],Agent watches another agent at a different cam...,50,2016,"[2004, 2010, 2009, 2011, 2005, 2011, 1992, 199..."


# 1. Similarity with Call for Papers

In [9]:
cp ../../rishi/abstract_jaccardCoef.pickle ./

In [10]:
with open("abstract_jaccardCoef.pickle", "rb") as f:
    abs_jc = pickle.load(f)

In [12]:
len(abs_jc), list(abs_jc.items())[0:3]

(4897,
 [('2020_ByeAK1BKPB', 0.024691358024691357),
  ('2020_BJe932EYwS', 0.03389830508474576),
  ('2017_Sk2iistgg', 0.05405405405405406)])

# 2. Text writing clarity features 
#noun_phrases in abstract and title \n
#nsubj, dobj in abstract
#AVERAGE(Non-stop word intersection between consecutive sentences in abstract)
#AVERAGE(cosine sim between consecutive sentences in abstract)

In [112]:
def text_clarity_features(title, abst):
    
    # a. np, nsunj, dobj in abstract
    abst = abst.strip("\n")
    abst = abst.strip()
    
    if not abst:
        doc_title = nlp(title)
        title_np = 0
        for tok in doc_title:
            if tok.pos_ == "NOUN":
                title_np += 1
        return [0, 0, 0], title_np, 0
    
    doc = nlp(abst)
    
    np_nsubj_dobj = [0, 0, 0]
    total_sents = 0
    
    for s in doc.sents:
        total_sents += 1
        for tok in s:
            if tok.pos_ == "NOUN":
                np_nsubj_dobj[0] = np_nsubj_dobj[0] + 1
            if tok.dep_.startswith("nsubj"):
                np_nsubj_dobj[1] = np_nsubj_dobj[1] + 1
            if tok.dep_.startswith("dobj"):
                np_nsubj_dobj[2] = np_nsubj_dobj[2] + 1
    
    np_nsubj_dobj = [np_nsubj_dobj[0]/total_sents, np_nsubj_dobj[1]/total_sents, np_nsubj_dobj[2]/total_sents]
    
    
    # b. np in title
    doc_title = nlp(title)
    title_np = 0
    for tok in doc_title:
        if tok.pos_ == "NOUN":
            title_np += 1
    
    
    # c. AVERAGE(Non-stop word intersection between consecutive sentences in abstract)
    intersection_list = []
    prev_sent_ent = []
    
    for s in doc.sents:
        curr_sent_ent = []
        for tok in s:
            if not tok.is_stop and not tok.pos_ == "PUNCT":
                curr_sent_ent.append(tok.lemma_)
        
        intersection_list.append(len(set(prev_sent_ent).intersection(set(curr_sent_ent))))
        prev_sent_ent = curr_sent_ent
    
    avg_intersection = np.mean(intersection_list[1:])
    
    return np_nsubj_dobj, title_np, avg_intersection

In [113]:
for index,row in text_data.iterrows():
    np_nsubj_dobj, title_np, avg_intersection = text_clarity_features(row["title"], row["abstract"])
    
    text_data.loc[index, "np_count_abs"] = np_nsubj_dobj[0]
    text_data.loc[index, "nsubj_count_abs"] = np_nsubj_dobj[1]
    text_data.loc[index, "dobj_count_abs"] = np_nsubj_dobj[2]
    text_data.loc[index, "np_count_title"] = title_np
    text_data.loc[index, "cons_inters_abs"] = avg_intersection
    
    text_data.loc[index, "jc_cfp"] = abs_jc[index]

/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [109]:
# text_data[text_data.isnull().any(axis=1)]

In [106]:
text_data[text_data["id"]=="2017_SJUdkecgx"]["abstract"].get_values()[0]

'  '

In [50]:
text_data.columns

Index(['cdate', 'content', 'ddate', 'details', 'forum', 'id', 'invitation',
       'label', 'nonreaders', 'number', 'original', 'readers', 'referent',
       'replyto', 'signatures', 'tcdate', 'tmdate', 'writers', 'title',
       'authors', 'abstract', 'keywords', 'tldr', 'ref_len', 'ref_latest',
       'ref_years', 'np_count_abs', 'nsubj_count_abs', 'dobj_count_abs',
       'np_count_title', 'cons_inters_abs', 'jc_cfp'],
      dtype='object')

In [115]:
def cosine_sim_consecutive(abst):
    
    sent_vects = []
    
    abst = abst.strip("\n")
    abst = abst.strip()
    if not abst:
        return 0
    
    doc = nlp(abst)
    
    for s in doc.sents:
        sent_vects.append(s.vector)
    
    cosine_sim_list = []
    
    for idx, vec in enumerate(sent_vects[:-1]):
        cosine_sim_list.append(1 - spatial.distance.cosine(vec, sent_vects[idx+1]))
    
    return np.mean(cosine_sim_list)

In [85]:
cosine_sim_consecutive(doc.text)

0.6028965943389468

In [116]:
for index,row in text_data.iterrows():
    text_data.loc[index, "abs_cons_cosine"] = cosine_sim_consecutive(row["abstract"])

In [117]:
text_data["abs_cons_cosine"].head(3)

2017_B1-Hhnslg    0.756707
2017_B1-q5Pqxl    0.602897
2017_B16dGcqlx    0.714312
Name: abs_cons_cosine, dtype: object

# 4. Leaderboard and Ablation table counts

In [51]:
ls ../features/

all_data_features_17_20.pkl
all_data_features_csv_17_20_new.pkl
all_data_features_csv_2017_new.pkl
all_data_features_csv_2017_with_aspect_scores.pkl
data_csv-2017-20.pkl
data_features_csv-2017.pkl
features_csv_17_20.pkl
GS_augmented_2017_iclr.pickle
GS_MMM_cit_pub_count_2017.pickle
iclr_arxiv_map.pkl
leaderboard_table_refs.pickle
leaderboard_table_refs.pkl
mag_features_2017_iclr.pickle
rev_aspects_2017_peerread.pkl


In [53]:
with open("../features/leaderboard_table_refs.pkl", "rb") as f:
    ldb = pickle.load(f)

In [54]:
with open("../features/leaderboard_table_refs.pickle", "rb") as f:
    ldb_other = pickle.load(f)

In [55]:
len(ldb), len(ldb_other)

(1988, 1988)

In [58]:
total_keys = [0, 0, 0, 0]

for k in ldb.keys():
    total_keys[0] = total_keys[0] + ldb[k]["tcount"][1]
    total_keys[1] = total_keys[1] + ldb[k]["tcount"][2]
    
    total_keys[2] = total_keys[2] + ldb_other[k]["tcount"][1]
    total_keys[3] = total_keys[3] + ldb_other[k]["tcount"][2]

In [59]:
total_keys

[3330, 3316, 3330, 3316]

In [60]:
for index,row in text_data.iterrows():
    if index in ldb.keys():
        text_data.loc[index, "ldb_table_count"] = ldb[index]["tcount"][1]
        text_data.loc[index, "abl_table_count"] = ldb[index]["tcount"][2]
    else:
        text_data.loc[index, "ldb_table_count"] = 0
        text_data.loc[index, "abl_table_count"] = 0

# 5. Save this for now

In [118]:
text_data.to_pickle("../features/writing_clarity_ldb_abl.pkl")

In [119]:
text_data["cons_inters_abs"].head(8)

2017_B1-Hhnslg    2.000000
2017_B1-q5Pqxl    0.777778
2017_B16dGcqlx    2.666667
2017_B184E5qee    2.000000
2017_B186cP9gx    1.500000
2017_B1E7Pwqgl    1.142857
2017_B1ElR4cgg    5.000000
2017_B1G9tvcgx    3.200000
Name: cons_inters_abs, dtype: float64